# 네이버 금융-> 투자정보 베너 크롤링

1) 시가총액  
2) 시가총액순위  
3) 상장주식수  
4) 액면가   
5) 매매단위  
6) 외국인한도주식수(A)  
7) 외국인보유주식수(B)   
8) 외국인소진율(B/A)     -> 최근 외국인 지분율  
9) 투자의견  
10) 목표주가  
11) 52주최고  
12) 52주최저  
13) PER{2020.09}  
14) EPS{2020.09}  
15) 추정PER  
16) 추정EPS  
17) PBR{2020.09}  
18) BPS{2020.09}  
19) 배당수익률  
20) 동일업종PER  
21) 동일업종등락률  

In [1]:
import sys
import os
import pandas as pd
import numpy as np

from selenium import webdriver
import time

# tqdm : for문 돌릴 때 진행상황
from tqdm import tqdm_notebook

# warning 메시지 무시하는 코드
import warnings ; warnings.filterwarnings(action='ignore')

In [2]:
# KOSPI 800.csv 에 추가로 붙이기
kospi_800 = pd.read_csv('KOSPI_800.csv')
kospi_800

,번호,종목코드,기업명,업종코드,업종,상장주식수(주),자본금(원),액면가(원),통화구분,대표전화,주소
0,1,95570,AJ네트웍스,147603,산업용 기계 및 장비 임대업,"46,822,295","46,822,295,000","1,000",원(KRW),02-6363-9999,"서울특별시 송파구 정의로8길 9 (문정동,AJ빌딩)"
1,2,6840,AK홀딩스,116409,기타 금융업,"13,247,561","66,237,805,000","5,000",원(KRW),02-768-2923,서울특별시 마포구 양화로 188 -
2,3,27410,BGF,116409,기타 금융업,"95,716,791","95,716,791,000","1,000",원(KRW),1577-3663,서울특별시 강남구 테헤란로 405
3,4,282330,BGF리테일,74701,종합 소매업,"17,283,906","17,283,906,000","1,000",원(KRW),02-1577-8007,서울특별시 강남구 테헤란로 405 BGF사옥
4,5,138930,BNK금융지주,116409,기타 금융업,"325,935,246","1,629,676,230,000","5,000",원(KRW),051-620-3023,부산광역시 남구 문현금융로 30(문현동)
...,...,...,...,...,...,...,...,...,...,...,...
795,796,79980,휴비스,32005,화학섬유 제조업,"34,500,000","172,500,000,000","5,000",원(KRW),02-2189-4567,서울특별시 강남구 학동로 343
796,797,5010,휴스틸,32401,1차 철강 제조업,"7,621,668","38,108,340,000","5,000",원(KRW),02-828-9000,"서울특별시 강남구 테헤란로 512 (대치동, 신안빌딩 14층)"
797,798,69260,휴켐스,32004,기타 화학제품 제조업,"40,878,588","40,878,588,000","1,000",원(KRW),02-2262-0600,서울특별시 중구 퇴계로 173 19층
798,799,540,흥국화재,116501,보험업,"64,242,645","325,821,225,000","5,000",원(KRW),02-2002-6000,서울특별시 종로구 새문안로 68


In [21]:
# 종목코드 6자리로 만들기
kospi_800 = kospi_800.astype({'종목코드': 'str'})

for i in range(800):
    fakecode = kospi_800['종목코드'][i]
    
    if len(fakecode)==2:
        rightcode = "0000"+ fakecode
        kospi_800['종목코드'][i] = rightcode
    
    if len(fakecode)==3:
        rightcode = "000"+ fakecode
        kospi_800['종목코드'][i] = rightcode
    if len(fakecode)==4:
        rightcode = "00"+ fakecode
        kospi_800['종목코드'][i] = rightcode    
    if len(fakecode)==5:
        rightcode = "0"+ fakecode
        kospi_800['종목코드'][i] = rightcode
        
kospi_800.head(50)

,번호,종목코드,기업명,업종코드,업종,상장주식수(주),자본금(원),액면가(원),통화구분,대표전화,주소
0,1,095570,AJ네트웍스,147603,산업용 기계 및 장비 임대업,"46,822,295","46,822,295,000","1,000",원(KRW),02-6363-9999,"서울특별시 송파구 정의로8길 9 (문정동,AJ빌딩)"
1,2,006840,AK홀딩스,116409,기타 금융업,"13,247,561","66,237,805,000","5,000",원(KRW),02-768-2923,서울특별시 마포구 양화로 188 -
2,3,027410,BGF,116409,기타 금융업,"95,716,791","95,716,791,000","1,000",원(KRW),1577-3663,서울특별시 강남구 테헤란로 405
3,4,282330,BGF리테일,74701,종합 소매업,"17,283,906","17,283,906,000","1,000",원(KRW),02-1577-8007,서울특별시 강남구 테헤란로 405 BGF사옥
4,5,138930,BNK금융지주,116409,기타 금융업,"325,935,246","1,629,676,230,000","5,000",원(KRW),051-620-3023,부산광역시 남구 문현금융로 30(문현동)
5,6,001460,BYC,31401,봉제의복 제조업,"624,615","4,200,000,000","5,000",원(KRW),02-840-3172,서울특별시 영등포구 도림천로21길 3
6,7,001040,CJ,116409,기타 금융업,"29,176,998","179,047,370,000","5,000",원(KRW),02-726-8114,"서울특별시 중구 소월로2길 12 (남대문로5가, CJ THE CENTER)"
7,8,079160,CJ CGV,105901,"영화, 비디오물, 방송프로그램 제작 및 배급업","35,100,000","17,550,000,000",500,원(KRW),02-371-6660,서울특별시 용산구 한강대로23길 55 아이파크몰 6층(한강로동)
8,9,000120,CJ대한통운,84903,도로 화물 운송업,"22,812,344","114,061,720,000","5,000",원(KRW),02-3782-0114,서울특별시 중구 세종대로9길 53
9,10,011150,CJ씨푸드,31007,기타 식품 제조업,"35,930,773","18,065,386,500",500,원(KRW),031-730-9114,경기도 성남시 중원구 둔촌대로388번길 32 (상대원동)


## [네이버 금융] 접속

In [14]:
# 한번에 편하게 추가하기 위해 리스트 생성
시가총액_리스트 = []
상장주식수_리스트 = []
액면가_리스트 = []
매매단위_리스트 = []
외국인한도주식수_리스트 = []
외국인보유주식수_리스트 = []
외국인소진율_리스트 = []
투자의견_리스트 = []
목표주가_리스트 = []
최고52주_리스트 = []
최저52주_리스트 = []
PER_202009_리스트 = []
EPS_202009_리스트 = []
추정PER_리스트 = []
추정EPS_리스트 = []
PBR_202009_리스트 = []
BPS_202009_리스트 = []
배당수익률_리스트 = []
동일업종PER_리스트 = []
동일업종등락률_리스트 = []

ERROR_list = []

In [41]:
for i in tqdm_notebook(range(648,800)):
    company = kospi_800['종목코드'][i]          # 종목코드
    
    #Step 1. 크롬 웹브라우저 실행
    path = "chromedriver.exe"  # 맥 기준  # 윈도우는 "chromedriver.exe"
    driver = webdriver.Chrome(path)
    
    # 사이트 주소는 네이버 금융
    driver.get('https://finance.naver.com/')
    time.sleep(2)
    
    #Step 2. 네이버 금융 검색창에 "회사 종목코드" 검색
    element = driver.find_element_by_id("stock_items")
    element.send_keys(company)
    element.submit()
    time.sleep(1)

    # 1) 시가총액 - " ___억원"
    시가총액 = driver.find_element_by_xpath("""//*[@id="_market_sum"]""").text
    시가총액_리스트.append(시가총액)

    # 3) 상장주식수
    try :
        상장주식수 = int(driver.find_element_by_xpath("""//*[@id="tab_con1"]/div[1]/table/tbody/tr[3]/td/em""").text.replace(',', ''))
    except : 
        상장주식수 = 'NA'
        ERROR_list.append(company)
    상장주식수_리스트.append(상장주식수)

    # 4) 액면가 - "__원"
    try :
        액면가 = int(driver.find_element_by_xpath("""//*[@id="tab_con1"]/div[1]/table/tbody/tr[4]/td/em[1]""").text.replace(',', ''))
    except : 액면가 = 'NA' 
    액면가_리스트.append(액면가)

    # 5) 매매단위 - "__주"
    try :
        매매단위 = int(driver.find_element_by_xpath("""//*[@id="tab_con1"]/div[1]/table/tbody/tr[4]/td/em[2]""").text)
    except : 매매단위 ='NA'
    매매단위_리스트.append(매매단위)

    # 6) 외국인한도주식수(A)
    try :
        외국인한도주식수 = int(driver.find_element_by_xpath("""//*[@id="tab_con1"]/div[2]/table/tbody/tr[1]/td/em""").text.replace(',', ''))
    except : 외국인한도주식수 ='NA'
    외국인한도주식수_리스트.append(외국인한도주식수)

    # 7) 외국인보유주식수(B)
    try : 
        외국인보유주식수 = int(driver.find_element_by_xpath("""//*[@id="tab_con1"]/div[2]/table/tbody/tr[2]/td/em""").text.replace(',', ''))
    except : 외국인보유주식수 ='NA'
    외국인보유주식수_리스트.append(외국인보유주식수)

    # 8) 외국인소진율(B/A) -> 최근 외국인 지분율 - "__.__%"
    try :
        외국인소진율 = float(driver.find_element_by_xpath("""//*[@id="tab_con1"]/div[2]/table/tbody/tr[3]/td/em""").text.replace('%', ''))
    except : 외국인소진율 ='NA'
    외국인소진율_리스트.append(외국인소진율)

    # 9) 투자의견 - "_.__ 매수"
    try :
        투자의견 = float(driver.find_element_by_xpath("""//*[@id="tab_con1"]/div[3]/table/tbody/tr[1]/td/span[1]/em""").text)
    except : 투자의견 = 'NA' 
    투자의견_리스트.append(투자의견)

    # 10) 목표주가 - "_____원"
    try :
        목표주가 = int(driver.find_element_by_xpath("""//*[@id="tab_con1"]/div[3]/table/tbody/tr[1]/td/em""").text.replace(',', ''))
    except : 목표주가 = 'NA' 
    목표주가_리스트.append(목표주가)

    # 11) 최고52주 - "_____원"
    try : 
        최고52주 = int(driver.find_element_by_xpath("""//*[@id="tab_con1"]/div[3]/table/tbody/tr[2]/td/em[1]""").text.replace(',', ''))
    except : 최고52주 = 'NA'
    최고52주_리스트.append(최고52주)

    # 12) 최저52주 - "_____원"
    try : 
        최저52주 = int(driver.find_element_by_xpath("""//*[@id="tab_con1"]/div[3]/table/tbody/tr[2]/td/em[2]""").text.replace(',', ''))
    except : 최저52주 = 'NA'
    최저52주_리스트.append(최저52주)

    # 13) PER{2020.09} - "__.__배"
    try :
        PER_202009 = float(driver.find_element_by_xpath("""//*[@id="_per"]""").text)
    except : PER_202009 = 'NA'
    PER_202009_리스트.append(PER_202009)

    # 14) EPS{2020.09} - "_____원"
    try :
        EPS_202009 = int(driver.find_element_by_xpath("""//*[@id="_eps"]""").text.replace(',', ''))
    except : EPS_202009 = 'NA'
    EPS_202009_리스트.append(EPS_202009)

    # 15) 추정PER - "__.__배"
    try :
        추정PER = float(driver.find_element_by_xpath("""//*[@id="_cns_per"]""").text)
    except : 추정PER = 'NA'
    추정PER_리스트.append(추정PER)

    # 16) 추정EPS - "_____원"
    try : 
        추정EPS = int(driver.find_element_by_xpath("""//*[@id="_cns_eps"]""").text.replace(',', ''))
    except : 추정EPS = 'NA'
    추정EPS_리스트.append(추정EPS)

    # 17) PBR{2020.09} - "__.__배"
    try :
        PBR_202009 = float(driver.find_element_by_xpath("""//*[@id="_pbr"]""").text)
    except : PBR_202009 = 'NA'
    PBR_202009_리스트.append(PBR_202009)

    # 18) BPS{2020.09} - "_____원"
    try :
        BPS_202009 = int(driver.find_element_by_xpath("""//*[@id="tab_con1"]/div[4]/table/tbody[2]/tr[2]/td/em[2]""").text.replace(',', ''))
    except : BPS_202009 = 'NA'
    BPS_202009_리스트.append(BPS_202009)

    # 19) 배당수익률
    try :
        배당수익률 = float(driver.find_element_by_xpath("""//*[@id="tab_con1"]/div[4]/table/tbody[2]/tr[3]/td/em""").text)
    except: 배당수익률 = 'NA'
    배당수익률_리스트.append(배당수익률)

    # 20) 동일업종PER - "__.__배"
    try :
        동일업종PER = float(driver.find_element_by_xpath("""//*[@id="tab_con1"]/div[5]/table/tbody/tr[1]/td/em""").text)
    except: 동일업종PER = 'NA'
    동일업종PER_리스트.append(동일업종PER)

    # 21) 동일업종등락률 - "__.__%"
    try : 
        동일업종등락률 = float(driver.find_element_by_xpath("""//*[@id="tab_con1"]/div[5]/table/tbody/tr[2]/td/em""").text.replace('%', ''))
    except: 동일업종등락률 = 'NA'
    동일업종등락률_리스트.append(동일업종등락률)
    
    driver.close()   

In [44]:
invest_data = pd.DataFrame(columns= ['시가총액', '상장주식수', '액면가', '매매단위','외국인한도주식수','외국인보유주식수',
                                    '외국인소진율', '투자의견','목표주가','최고52주','최저52주','PER_202009','EPS_202009',
                                    '추정PER','추정EPS','PBR_202009','BPS_202009','배당수익률','동일업종PER','동일업종등락률'])

invest_data['시가총액']= 시가총액_리스트
invest_data['상장주식수']= 상장주식수_리스트
invest_data['액면가']= 액면가_리스트
invest_data['매매단위']=매매단위_리스트
invest_data['외국인한도주식수']=외국인한도주식수_리스트
invest_data['외국인보유주식수']=외국인보유주식수_리스트
invest_data['외국인소진율']=외국인소진율_리스트
invest_data['투자의견']=투자의견_리스트
invest_data['목표주가']=목표주가_리스트
invest_data['최고52주']=최고52주_리스트
invest_data['PER_202009']=PER_202009_리스트
invest_data['EPS_202009']=EPS_202009_리스트
invest_data['추정PER']=추정PER_리스트
invest_data['추정EPS']=추정EPS_리스트
invest_data['PBR_202009']=PBR_202009_리스트
invest_data['BPS_202009']=BPS_202009_리스트
invest_data['배당수익률']=배당수익률_리스트
invest_data['동일업종PER']=동일업종PER_리스트
invest_data['동일업종등락률']=동일업종등락률_리스트

invest_data

,시가총액,상장주식수,액면가,매매단위,외국인한도주식수,외국인보유주식수,외국인소진율,투자의견,목표주가,최고52주,최저52주,PER_202009,EPS_202009,추정PER,추정EPS,PBR_202009,BPS_202009,배당수익률,동일업종PER,동일업종등락률
0,"2,091",46822295,1000,1,46822295,4980434,10.64,NA,NA,5270,NaN,NA,-1230,NA,NA,0.69,6446,NA,51.92,1.31
1,"3,398",13247561,5000,1,13247561,1013755,7.65,NA,NA,34500,NaN,NA,-11544,NA,NA,0.49,52837,NA,63.27,3.47
2,"4,695",95716791,1000,1,95716791,7666167,8.01,4,6000,5520,NaN,18.44,266,NA,NA,0.31,15772,NA,-27.68,2
3,"2조 5,062",17283906,1000,1,17283906,5408587,31.29,3.77,172455,175500,NaN,19.15,7570,19.15,7570,3.73,38910,NA,-27.68,2
4,"1조 9,067",325935246,5000,1,325935246,124968881,38.34,3.8,7207,7290,NaN,3.97,1474,3.75,1562,0.22,26773,NA,4.89,0.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,"2,732",34500000,5000,1,34500000,1214235,3.52,4,11000,9850,NaN,10.25,773,NA,NA,0.62,12821,NA,30.14,2.17
796,590,7621668,5000,1,7621668,228865,3,NA,NA,9830,NaN,NA,-2448,NA,NA,0.15,51984,NA,41.1,2.67
797,1조 77,40878588,1000,1,40878588,3501873,8.57,3.75,30500,27650,NaN,20.91,1179,13.79,1788,1.42,17352,NA,63.27,3.47
798,"2,101",64242645,5000,1,64242645,1233684,1.92,NA,NA,4290,NaN,14.16,231,NA,NA,0.28,11810,NA,7.1,0.65


In [48]:
invest_data.to_csv("invest_information_800.csv", encoding= 'utf-8-sig', index=False)

In [45]:
ERROR_list

['365550',
 '338100',
 '330590',
 '088980',
 '094800',
 '204210',
 '357250',
 '155900',
 '096300',
 '293940',
 '140910',
 '088260',
 '350520',
 '334890',
 '348950',
 '145270',
 '357120',
 '172580',
 '153360',
 '152550',
 '168490']

In [46]:
# 제대로 획득이 안 된 종목코드 개수
len(ERROR_list)

21

In [54]:
# 가져 올 수 있는게,,,, 시가총액, 상장주식수, 최고52주, 최저52주

for i in tqdm_notebook(ERROR_list):
    company = i       # 종목코드
    
    #Step 1. 크롬 웹브라우저 실행
    path = "chromedriver.exe"  # 맥 기준  # 윈도우는 "chromedriver.exe"
    driver = webdriver.Chrome(path)
    
    # 사이트 주소는 네이버 금융
    driver.get('https://finance.naver.com/')
    time.sleep(2)
    
    #Step 2. 네이버 금융 검색창에 "회사 종목코드" 검색
    element = driver.find_element_by_id("stock_items")
    element.send_keys(company)
    element.submit()
    time.sleep(1)

    # 1) 시가총액 - " ___억원"
    시가총액_에러 = driver.find_element_by_xpath("""//*[@id="_market_sum"]""").text
    invest_data[invest_data['종목코드']== i]['시가총액'] = 시가총액_에러 
    
    # 3) 상장주식수
    상장주식수_에러 = int(driver.find_element_by_xpath("""//*[@id="tab_con1"]/div[1]/table/tbody/tr[2]/td/em""").text.replace(',', ''))
    invest_data[invest_data['종목코드']== i]['상장주식수'] = 상장주식수_에러 
    
    # 11) 최고52주 - "_____원"
    최고52주_에러 = int(driver.find_element_by_xpath("""//*[@id="tab_con1"]/div[2]/table/tbody/tr/td/em[1]""").text.replace(',', ''))
    invest_data[invest_data['종목코드']== i]['최고52주'] = 최고52주_에러
    
    # 12) 최저52주 - "_____원"
    최저52주_에러 = int(driver.find_element_by_xpath("""//*[@id="tab_con1"]/div[2]/table/tbody/tr/td/em[2]""").text.replace(',', ''))
    invest_data[invest_data['종목코드']== i]['최저52주'] = 최저52주_에러

    
    driver.close()   

In [49]:
int('365550')

365550

In [50]:
invest_data['종목코드'] = kospi_800['종목코드']
invest_data

,시가총액,상장주식수,액면가,매매단위,외국인한도주식수,외국인보유주식수,외국인소진율,투자의견,목표주가,최고52주,...,PER_202009,EPS_202009,추정PER,추정EPS,PBR_202009,BPS_202009,배당수익률,동일업종PER,동일업종등락률,종목코드
0,"2,091",46822295,1000,1,46822295,4980434,10.64,NA,NA,5270,...,NA,-1230,NA,NA,0.69,6446,NA,51.92,1.31,095570
1,"3,398",13247561,5000,1,13247561,1013755,7.65,NA,NA,34500,...,NA,-11544,NA,NA,0.49,52837,NA,63.27,3.47,006840
2,"4,695",95716791,1000,1,95716791,7666167,8.01,4,6000,5520,...,18.44,266,NA,NA,0.31,15772,NA,-27.68,2,027410
3,"2조 5,062",17283906,1000,1,17283906,5408587,31.29,3.77,172455,175500,...,19.15,7570,19.15,7570,3.73,38910,NA,-27.68,2,282330
4,"1조 9,067",325935246,5000,1,325935246,124968881,38.34,3.8,7207,7290,...,3.97,1474,3.75,1562,0.22,26773,NA,4.89,0.72,138930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,"2,732",34500000,5000,1,34500000,1214235,3.52,4,11000,9850,...,10.25,773,NA,NA,0.62,12821,NA,30.14,2.17,079980
796,590,7621668,5000,1,7621668,228865,3,NA,NA,9830,...,NA,-2448,NA,NA,0.15,51984,NA,41.1,2.67,005010
797,1조 77,40878588,1000,1,40878588,3501873,8.57,3.75,30500,27650,...,20.91,1179,13.79,1788,1.42,17352,NA,63.27,3.47,069260
798,"2,101",64242645,5000,1,64242645,1233684,1.92,NA,NA,4290,...,14.16,231,NA,NA,0.28,11810,NA,7.1,0.65,000540


In [53]:
invest_data[invest_data['종목코드']== '095570']['시가총액'] = 

0    2,091
Name: 시가총액, dtype: object

In [66]:
invest_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 21 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   시가총액        800 non-null    object
 1   상장주식수       800 non-null    object
 2   액면가         800 non-null    object
 3   매매단위        800 non-null    object
 4   외국인한도주식수    800 non-null    int64 
 5   외국인보유주식수    800 non-null    object
 6   외국인소진율      800 non-null    object
 7   투자의견        800 non-null    object
 8   목표주가        800 non-null    object
 9   최고52주       800 non-null    object
 10  최저52주       0 non-null      object
 11  PER_202009  800 non-null    object
 12  EPS_202009  800 non-null    object
 13  추정PER       800 non-null    object
 14  추정EPS       800 non-null    object
 15  PBR_202009  800 non-null    object
 16  BPS_202009  800 non-null    object
 17  배당수익률       800 non-null    object
 18  동일업종PER     800 non-null    object
 19  동일업종등락률     800 non-null    object
 20  종목코드      

In [65]:
invest_data[invest_data['종목코드']== '365550']

,시가총액,상장주식수,액면가,매매단위,외국인한도주식수,외국인보유주식수,외국인소진율,투자의견,목표주가,최고52주,...,PER_202009,EPS_202009,추정PER,추정EPS,PBR_202009,BPS_202009,배당수익률,동일업종PER,동일업종등락률,종목코드
21,"7,607",NA,NA,NA,5320,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,365550


In [68]:
for i in ERROR_list:
    
    print(i, invest_data[invest_data['종목코드']== i].index)
    #Step 1. 크롬 웹브라우저 실행
    path = "chromedriver.exe"  # 맥 기준  # 윈도우는 "chromedriver.exe"
    driver = webdriver.Chrome(path)

    # 사이트 주소는 네이버 금융
    driver.get('https://finance.naver.com/')
    time.sleep(2)

    #Step 2. 네이버 금융 검색창에 "회사 종목코드" 검색
    element = driver.find_element_by_id("stock_items")
    element.send_keys(i)
    element.submit()
    time.sleep(1)

    # 1) 시가총액 - " ___억원"
    시가총액_에러 = driver.find_element_by_xpath("""//*[@id="_market_sum"]""").text
    invest_data[invest_data['종목코드']== i]['시가총액'] = 시가총액_에러 
    print(시가총액_에러)

    # 3) 상장주식수
    상장주식수_에러 = int(driver.find_element_by_xpath("""//*[@id="tab_con1"]/div[1]/table/tbody/tr[2]/td/em""").text.replace(',', ''))
    invest_data[invest_data['종목코드']== i]['상장주식수'] = 상장주식수_에러
    print(상장주식수_에러)

    # 11) 최고52주 - "_____원"
    최고52주_에러 = int(driver.find_element_by_xpath("""//*[@id="tab_con1"]/div[2]/table/tbody/tr/td/em[1]""").text.replace(',', ''))
    invest_data[invest_data['종목코드']== i]['최고52주'] = 최고52주_에러
    print(최고52주_에러)

    # 12) 최저52주 - "_____원"
    최저52주_에러 = int(driver.find_element_by_xpath("""//*[@id="tab_con1"]/div[2]/table/tbody/tr/td/em[2]""").text.replace(',', ''))
    invest_data[invest_data['종목코드']== i]['최저52주'] = 최저52주_에러
    print(최저52주_에러)
    print('\n')

365550 Int64Index([21], dtype='int64')
7,607
143259000
5320
4800


338100 Int64Index([70], dtype='int64')
781
18660000
5980
4100


330590 Int64Index([257], dtype='int64')
8,822
171968884
5930
4435


088980 Int64Index([271], dtype='int64')
3조 9,266
372192485
11950
9260


094800 Int64Index([272], dtype='int64')
3,583
92834331
4890
3150


204210 Int64Index([281], dtype='int64')
236
7826815
3620
2125


357250 Int64Index([290], dtype='int64')
945
20100000
5050
4300


155900 Int64Index([297], dtype='int64')
344
15200004
2925
2250


096300 Int64Index([302], dtype='int64')
326
137680211
360
231


293940 Int64Index([421], dtype='int64')
3,606
52330123
7694
5576


140910 Int64Index([455], dtype='int64')
256
3792775
8719
4508


088260 Int64Index([516], dtype='int64')
3,243
63341590
6400
4305


350520 Int64Index([525], dtype='int64')
989
20600000
5130
4060


334890 Int64Index([526], dtype='int64')
1,132
24000000
5030
4375


348950 Int64Index([552], dtype='int64')
8,528
165600000
5400
4600


145270